This script retrieves data on existing hydrogen plants and calculates their annual hydrogen production (using 2022 data from the EPA GHGRP). Subpart P Hydrogen Production emissions (in metric tons) are converted to hydrogen production totals (in kg) using the ratio of 9.0 kg CO2 per kg H2 (taken from https://pubs.acs.org/doi/full/10.1021/acs.est.8b06197).

Outputs to a folder titled 'wecc_existing_h2_plants' under the industry processing outputs. The output .csv file can be directly pasted into the industry model inputs folder.

In [5]:
import pandas as pd
import geopandas as gpd
from pathlib import Path

# Constant conversion
kg_CO2_per_kg_h2 = 9.0

# File paths
base_path = Path.cwd()

inputs_path = base_path / 'industry_processing_inputs'
load_zone_path = inputs_path / 'load_zones/load_zones.shp'
ghgrp_path =  inputs_path / 'ghgrp_data_2022.xlsx'

output_path = base_path / 'industry_processing_outputs' / 'wecc_existing_h2_plants'
output_path.mkdir(parents=True, exist_ok=True)

In [6]:

# Read and filter hydrogen production sites
ghgrp_df = pd.read_excel(ghgrp_path, header=3)
hydrogen_df = ghgrp_df[~ghgrp_df['Hydrogen Production'].isna()]

# Convert to GeoDataFrame
hydrogen_gdf = gpd.GeoDataFrame(
    hydrogen_df,
    geometry=gpd.points_from_xy(hydrogen_df['Longitude'], hydrogen_df['Latitude']),
    crs="EPSG:4326"
)

# Read load zones and ensure same CRS
load_zones = gpd.read_file(load_zone_path).to_crs("EPSG:4326")

# Spatial join to find hydrogen sites in load zones
wecc_hydrogen_df = gpd.sjoin(hydrogen_gdf, load_zones, how="inner", predicate="intersects")

# Estimate 2023 hydrogen demand from each facility by converting from mt CO2 to kg H2
wecc_hydrogen_df['hydrogen_demand_kg'] = wecc_hydrogen_df['Hydrogen Production'] * 1000 / kg_CO2_per_kg_h2

wecc_hydrogen_df.to_file(output_path / 'existing_h2_plants_wecc_2022.gpkg',driver='GPKG')

wecc_hydrogen_df = wecc_hydrogen_df.drop(columns=['geometry', 'index_right'])

wecc_hydrogen_df.to_csv(output_path / 'wecc_existing_h2_plants_2022.csv',index=False)